In [67]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

import plotly.offline as py
import plotly.graph_objects as go
import random

In [68]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.options.display.max_colwidth = 2000

### Read in cleaned and tokenized 311 HPD claims

In [69]:
df_311_wide = pd.read_csv("data/HPD_311_all_cleaned_tagged.csv", dtype = {'incident_zip_str':'str', }, parse_dates = ['created_date', 'closed_date', 'resolution_action_updated_date'])
print(df_311_wide.shape)

(6122816, 64)


In [70]:
#Creating separate dataset to analyze so that if I later want to drop anything I easily can
to_analyze = df_311_wide.copy()

## Part 1: Create white Sankey diagram on black background

### Create source/target dataframe using complaint_type variables and process and outcome tokens as nodes
Each of these things will be used as a layer in the sankey diagram

In [95]:
#Grouped process and outcome tags
df_by_complaint_process = pd.DataFrame(to_analyze.groupby(['grouped_complaint_type','grouped_ultimate_process'])['unique_key'].nunique()).reset_index()
df_by_complaint_process.columns = ['source','target', 'num_records']
display(df_by_complaint_process.head())

df_by_process_outcome = pd.DataFrame(to_analyze.groupby(['grouped_ultimate_process','grouped_ultimate_outcome'])['unique_key'].nunique()).reset_index()
df_by_process_outcome.columns = ['source','target', 'num_records']
display(df_by_complaint_process.head())

#Concatenate each set of connections
edges_df = pd.concat([df_by_complaint_process, df_by_process_outcome], ignore_index = True)
display(edges_df)

,source,target,num_records
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35
3,APPLIANCE,CONTACTED TENANT,5049
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985


,source,target,num_records
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35
3,APPLIANCE,CONTACTED TENANT,5049
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985


,source,target,num_records
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35
3,APPLIANCE,CONTACTED TENANT,5049
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985
5,APPLIANCE,RESOLUTION PROCESS UNKNOWN,2472
6,CONSTRUCTION,ATTEMPTED AND FAILED INSPECTION,114333
7,CONSTRUCTION,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35
8,CONSTRUCTION,CONTACTED TENANT,10755
9,CONSTRUCTION,INSPECTED ORIGINAL PROPERTY,346818


### Create a node dict with the type of node (complaint, process, or outcome)

In [96]:
#Create a node dict
df_by_complaint_process_outcome = pd.DataFrame(to_analyze.groupby(['grouped_complaint_type','grouped_ultimate_process', 'grouped_ultimate_outcome'])['unique_key'].nunique()).reset_index()
df_by_complaint_process_outcome.columns = ['grouped_complaint_type', 'grouped_ultimate_process', 'grouped_ultimate_outcome', 'num_records']
display(df_by_complaint_process_outcome.head())

tokens = list(df_by_complaint_process_outcome['grouped_ultimate_process'].unique())
node_type = ['process']*len((list(df_by_complaint_process_outcome['grouped_ultimate_process'].unique())))
                   
tokens.extend(list(df_by_complaint_process_outcome['grouped_ultimate_outcome'].unique()))
node_type.extend(['outcome']*len((list(df_by_complaint_process_outcome['grouped_ultimate_outcome'].unique()))))
                   
token_dict = pd.DataFrame({'node':tokens, 'node_type':node_type})

#Add in all unique complaint types to create node_dict
complaint_type = list(df_by_complaint_process_outcome['grouped_complaint_type'].unique())
node_type = ['complaint_type']*len((list(df_by_complaint_process_outcome['grouped_complaint_type'].unique())))

complaint_dict = pd.DataFrame({'node':complaint_type, 'node_type':node_type})

#Conatenate into one node dict
node_dict = pd.concat([complaint_dict, token_dict], ignore_index = True)

display(node_dict)

,grouped_complaint_type,grouped_ultimate_process,grouped_ultimate_outcome,num_records
0,AGENCY,INSPECTED ORIGINAL PROPERTY,NO VIOLATIONS WERE ISSUED,6
1,AGENCY,INSPECTED ORIGINAL PROPERTY,VIOLATIONS WERE ISSUED,3
2,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,COMPLAINT STILL OPEN,30958
3,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),OUTCOME UNKNOWN CLOSED,35
4,APPLIANCE,CONTACTED TENANT,CONDITIONS CORRECTED,5049


,node,node_type
0,AGENCY,complaint_type
1,APPLIANCE,complaint_type
2,CONSTRUCTION,complaint_type
3,DOOR/WINDOW,complaint_type
4,ELECTRIC,complaint_type
5,ELEVATOR,complaint_type
6,FLOORING/STAIRS,complaint_type
7,GENERAL,complaint_type
8,HEAT/HOT WATER,complaint_type
9,HPD LITERATURE REQUEST,complaint_type


### Put source/target in dictionary container that will be used to create sankey diagram

In [97]:
#Initialize dictionary to hold data for sankey diagram
sankey_info = {}

#Assign indices to each node for source/target labeling
node_dict['node_index'] = range(node_dict.shape[0])
sankey_info['data'] = {}
sankey_info['data']['node'] = {}
sankey_info['data']['node']['label'] = list(node_dict['node'].values)

#Merge in the node indices to the edges_df
edges_with_node_indices = edges_df.merge(node_dict, left_on='source', right_on = 'node', how = 'left')
edges_with_node_indices.rename(columns = {'node_index':'source_index', 'node_type':'source_node_type'}, inplace = True)
edges_with_node_indices.drop(columns = 'node', inplace = True)

edges_with_node_indices = edges_with_node_indices.merge(node_dict, left_on='target', right_on = 'node', how = 'left')
edges_with_node_indices.rename(columns = {'node_index':'target_index', 'node_type':'target_node_type'}, inplace = True)
edges_with_node_indices.drop(columns = 'node', inplace = True)

display(edges_with_node_indices)

#Use assigned indices to define source/target lists for diagram
sankey_info['data']['link'] = {}
sankey_info['data']['link']['source'] = list(edges_with_node_indices['source_index'].values)
sankey_info['data']['link']['target'] = list(edges_with_node_indices['target_index'].values)
sankey_info['data']['link']['value'] = list(edges_with_node_indices['num_records'].values)


,source,target,num_records,source_node_type,source_index,target_node_type,target_index
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9,complaint_type,0,process,17
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958,complaint_type,1,process,18
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35,complaint_type,1,process,19
3,APPLIANCE,CONTACTED TENANT,5049,complaint_type,1,process,20
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985,complaint_type,1,process,17
5,APPLIANCE,RESOLUTION PROCESS UNKNOWN,2472,complaint_type,1,process,21
6,CONSTRUCTION,ATTEMPTED AND FAILED INSPECTION,114333,complaint_type,2,process,18
7,CONSTRUCTION,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35,complaint_type,2,process,19
8,CONSTRUCTION,CONTACTED TENANT,10755,complaint_type,2,process,20
9,CONSTRUCTION,INSPECTED ORIGINAL PROPERTY,346818,complaint_type,2,process,17


### Define node/edge colors

In [98]:
sankey_info['data']['node']['color'] = ["rgba(255, 255, 255, 1)" for i in range(len(sankey_info['data']['node']['label']))]
sankey_info['data']['link']['color'] = ["rgba(255, 255, 255, .5)" for i in range(len(sankey_info['data']['link']['value']))]


### Create diagram with plotly package

In [103]:
fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "TWh",
    # Define nodes
    node = dict(
      pad = 50,
      thickness = 15,
      line = dict(color = "white", width = 0.5),
      label =  sankey_info['data']['node']['label'],
      color =  sankey_info['data']['node']['color']
    ),
    # Add links
    link = dict(
      source =  sankey_info['data']['link']['source'],
      target =  sankey_info['data']['link']['target'],
      value =  sankey_info['data']['link']['value'],
      color =  sankey_info['data']['link']['color']
))])

fig.update_layout(title_text="NYC HPD 311 complaints resolution process<br>Data source: NYC Open Data Portal",
                  font=dict(size = 7, color = 'white'),
                  plot_bgcolor='black',
                  paper_bgcolor='black',
                  title_font_size = 15)
fig.show()
fig.write_html('output/sankey_black_and_white.html')

## Part 2: Create second sankey diagram colored by complaint type


### New source/target dataframe with complaint type merged into all nodes in second layer for coloring

In [104]:
#First set of links is the same complaint_type/process set
display(df_by_complaint_process.head())

#Second set of links requires to group by complaint_type too
df_by_complaint_process_outcome = pd.DataFrame(to_analyze.groupby(['grouped_complaint_type','grouped_ultimate_process', 'grouped_ultimate_outcome'])['unique_key'].nunique()).reset_index()
df_by_complaint_process_outcome.columns = ['complaint_type_for_level_2', 'source', 'target', 'num_records']
display(df_by_complaint_process_outcome.head())

#Concatenate into one edges_df
edges_df = pd.concat([df_by_complaint_process, df_by_complaint_process_outcome], ignore_index = True, sort = False)
display(edges_df.head())

,source,target,num_records
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35
3,APPLIANCE,CONTACTED TENANT,5049
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985


,complaint_type_for_level_2,source,target,num_records
0,AGENCY,INSPECTED ORIGINAL PROPERTY,NO VIOLATIONS WERE ISSUED,6
1,AGENCY,INSPECTED ORIGINAL PROPERTY,VIOLATIONS WERE ISSUED,3
2,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,COMPLAINT STILL OPEN,30958
3,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),OUTCOME UNKNOWN CLOSED,35
4,APPLIANCE,CONTACTED TENANT,CONDITIONS CORRECTED,5049


,source,target,num_records,complaint_type_for_level_2
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9,NaN
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958,NaN
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35,NaN
3,APPLIANCE,CONTACTED TENANT,5049,NaN
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985,NaN


### Create a node dict

In [105]:
df_by_complaint_process_outcome = pd.DataFrame(to_analyze.groupby(['grouped_complaint_type','grouped_ultimate_process', 'grouped_ultimate_outcome'])['unique_key'].nunique()).reset_index()
df_by_complaint_process_outcome.columns = ['grouped_complaint_type', 'grouped_ultimate_process', 'grouped_ultimate_outcome', 'num_records']
display(df_by_complaint_process_outcome.head())

tokens = list(df_by_complaint_process_outcome['grouped_ultimate_process'].unique())
node_type = ['process']*len((list(df_by_complaint_process_outcome['grouped_ultimate_process'].unique())))
                   
tokens.extend(list(df_by_complaint_process_outcome['grouped_ultimate_outcome'].unique()))
node_type.extend(['outcome']*len((list(df_by_complaint_process_outcome['grouped_ultimate_outcome'].unique()))))
                   
token_dict = pd.DataFrame({'node':tokens, 'node_type':node_type})

#Add in all unique complaint types to create node_dict
complaint_type = list(df_by_complaint_process_outcome['grouped_complaint_type'].unique())
node_type = ['complaint_type']*len((list(df_by_complaint_process_outcome['grouped_complaint_type'].unique())))

complaint_dict = pd.DataFrame({'node':complaint_type, 'node_type':node_type})

#Conatenate into one node dict
node_dict = pd.concat([complaint_dict, token_dict], ignore_index = True)

display(node_dict)

,grouped_complaint_type,grouped_ultimate_process,grouped_ultimate_outcome,num_records
0,AGENCY,INSPECTED ORIGINAL PROPERTY,NO VIOLATIONS WERE ISSUED,6
1,AGENCY,INSPECTED ORIGINAL PROPERTY,VIOLATIONS WERE ISSUED,3
2,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,COMPLAINT STILL OPEN,30958
3,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),OUTCOME UNKNOWN CLOSED,35
4,APPLIANCE,CONTACTED TENANT,CONDITIONS CORRECTED,5049


,node,node_type
0,AGENCY,complaint_type
1,APPLIANCE,complaint_type
2,CONSTRUCTION,complaint_type
3,DOOR/WINDOW,complaint_type
4,ELECTRIC,complaint_type
5,ELEVATOR,complaint_type
6,FLOORING/STAIRS,complaint_type
7,GENERAL,complaint_type
8,HEAT/HOT WATER,complaint_type
9,HPD LITERATURE REQUEST,complaint_type


### Organize into container for sankey diagram

In [106]:
#Initialize dictionary to hold data for sankey diagram
sankey_info = {}

#Assign indices to each node for source/target labeling
node_dict['node_index'] = range(node_dict.shape[0])
sankey_info['data'] = {}
sankey_info['data']['node'] = {}
sankey_info['data']['node']['label'] = list(node_dict['node'].values)

#Merge in the node indices to the edges_df
edges_with_node_indices = edges_df.merge(node_dict, left_on='source', right_on = 'node', how = 'left')
edges_with_node_indices.rename(columns = {'node_index':'source_index', 'node_type':'source_node_type'}, inplace = True)
edges_with_node_indices.drop(columns = 'node', inplace = True)

edges_with_node_indices = edges_with_node_indices.merge(node_dict, left_on='target', right_on = 'node', how = 'left')
edges_with_node_indices.rename(columns = {'node_index':'target_index', 'node_type':'target_node_type'}, inplace = True)
edges_with_node_indices.drop(columns = 'node', inplace = True)

display(edges_with_node_indices)

#Use assigned indices to define source/target lists for diagram
sankey_info['data']['link'] = {}
sankey_info['data']['link']['source'] = list(edges_with_node_indices['source_index'].values)
sankey_info['data']['link']['target'] = list(edges_with_node_indices['target_index'].values)
sankey_info['data']['link']['value'] = list(edges_with_node_indices['num_records'].values)


,source,target,num_records,complaint_type_for_level_2,source_node_type,source_index,target_node_type,target_index
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9,NaN,complaint_type,0,process,17
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958,NaN,complaint_type,1,process,18
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35,NaN,complaint_type,1,process,19
3,APPLIANCE,CONTACTED TENANT,5049,NaN,complaint_type,1,process,20
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985,NaN,complaint_type,1,process,17
5,APPLIANCE,RESOLUTION PROCESS UNKNOWN,2472,NaN,complaint_type,1,process,21
6,CONSTRUCTION,ATTEMPTED AND FAILED INSPECTION,114333,NaN,complaint_type,2,process,18
7,CONSTRUCTION,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35,NaN,complaint_type,2,process,19
8,CONSTRUCTION,CONTACTED TENANT,10755,NaN,complaint_type,2,process,20
9,CONSTRUCTION,INSPECTED ORIGINAL PROPERTY,346818,NaN,complaint_type,2,process,17


### Color by complaint type
Uncomment this if you want to assign different colors to each category
This is what created the colors.csv ordering loaded in below

In [107]:
# #Assign colors to nodes and links such that they are colored according to complaint type

# #Define colors to use
# twenty_distinct_colors = ['(230, 25, 75)', '(60, 180, 75)', '(255, 225, 25)', '(0, 130, 200)', '(245, 130, 48)', '(145, 30, 180)', '(70, 240, 240)', '(240, 50, 230)', '(210, 245, 60)', '(250, 190, 212)', '(0, 128, 128)', '(220, 190, 255)', '(170, 110, 40)', '(255, 250, 200)', '(128, 0, 0)', '(170, 255, 195)', '(128, 128, 0)', '(255, 215, 180)', '(0, 0, 128)', '(128, 128, 128)', '(255, 255, 255)', '(0, 0, 0)']

# #Omit black and gray (so actually 18)
# distinct_colors = ['(230, 25, 75)', '(60, 180, 75)', '(255, 225, 25)', '(0, 130, 200)', '(245, 130, 48)', '(145, 30, 180)', '(70, 240, 240)', '(240, 50, 230)', '(210, 245, 60)', '(250, 190, 212)', '(0, 128, 128)', '(220, 190, 255)', '(170, 110, 40)', '(255, 250, 200)', '(128, 0, 0)', '(170, 255, 195)', '(128, 128, 0)', '(255, 215, 180)', '(0, 0, 128)', '(255, 255, 255)']

# #Randomly shuffle colors
# random.shuffle(distinct_colors)

# #Export this orientation of colors out
# # pd.DataFrame({'color':distinct_colors}).to_csv('colors.csv')

In [108]:
#Read in the previously defined orientation of colors
colors = pd.read_csv("colors.csv")
distinct_colors = list(colors['color'].values)
print(distinct_colors)

['(128, 128, 0)', '(250, 190, 212)', '(255, 250, 200)', '(240, 50, 230)', '(170, 110, 40)', '(60, 180, 75)', '(145, 30, 180)', '(70, 240, 240)', '(0, 128, 128)', '(0, 130, 200)', '(220, 190, 255)', '(230, 25, 75)', '(245, 130, 48)', '(0, 0, 128)', '(255, 215, 180)', '(170, 255, 195)', '(255, 225, 25)', '(128, 0, 0)', '(255, 255, 255)', '(210, 245, 60)']


In [109]:
#Merge in colors to complaint dict
complaint_dict['color'] = distinct_colors[0:complaint_dict.shape[0]]

#Use complaint dict to set edge colors
#Separate by layer
source_complaint_type = edges_with_node_indices[edges_with_node_indices['source_node_type'] == 'complaint_type']
source_process = edges_with_node_indices[edges_with_node_indices['source_node_type'] == 'process']

#Merge in colors to complaint type source
source_complaint_type_with_colors = source_complaint_type.merge(complaint_dict[['node', 'color']], left_on = 'source', right_on = 'node', how = 'left')
source_complaint_type_with_colors.drop(columns = 'node', inplace = True)
display(source_complaint_type_with_colors.head())

#Use complaint type to color second layer
source_process_with_colors = source_process.merge(complaint_dict[['node', 'color']], left_on = 'complaint_type_for_level_2', right_on = 'node', how = 'left')
display(source_process_with_colors.head())

#Concatenate edges back together
edges_with_colors = pd.concat([source_complaint_type_with_colors, source_process_with_colors], ignore_index = True, sort = False)

#Add opacity for edges and nodes
edges_with_colors['color'] = edges_with_colors['color'].apply(lambda color: 'rgba' + color.split(')')[0] + ', .6)')

display(edges_with_colors.head())

#Define node/edge colors
# sankey_info['data']['node']['color'] = ["rgba(255, 255, 255, .8)" for i in range(len(sankey_info['data']['node']['label']))]
sankey_info['data']['link']['color'] = list(edges_with_colors['color'].values)

#Define complaint_type for hovertemplate
edges_with_colors['complaint_type'] = edges_with_colors.apply(lambda row: row['source'] if row['source_node_type'] =='complaint_type' else row['complaint_type_for_level_2'], axis = 1)
sankey_info['data']['link']['label'] = list(edges_with_colors['complaint_type'].values)


,source,target,num_records,complaint_type_for_level_2,source_node_type,source_index,target_node_type,target_index,color
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9,NaN,complaint_type,0,process,17,"(128, 128, 0)"
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958,NaN,complaint_type,1,process,18,"(250, 190, 212)"
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35,NaN,complaint_type,1,process,19,"(250, 190, 212)"
3,APPLIANCE,CONTACTED TENANT,5049,NaN,complaint_type,1,process,20,"(250, 190, 212)"
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985,NaN,complaint_type,1,process,17,"(250, 190, 212)"


,source,target,num_records,complaint_type_for_level_2,source_node_type,source_index,target_node_type,target_index,node,color
0,INSPECTED ORIGINAL PROPERTY,NO VIOLATIONS WERE ISSUED,6,AGENCY,process,17,outcome,24,AGENCY,"(128, 128, 0)"
1,INSPECTED ORIGINAL PROPERTY,VIOLATIONS WERE ISSUED,3,AGENCY,process,17,outcome,25,AGENCY,"(128, 128, 0)"
2,ATTEMPTED AND FAILED INSPECTION,COMPLAINT STILL OPEN,30958,APPLIANCE,process,18,outcome,26,APPLIANCE,"(250, 190, 212)"
3,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),OUTCOME UNKNOWN CLOSED,35,APPLIANCE,process,19,outcome,27,APPLIANCE,"(250, 190, 212)"
4,CONTACTED TENANT,CONDITIONS CORRECTED,5049,APPLIANCE,process,20,outcome,28,APPLIANCE,"(250, 190, 212)"


,source,target,num_records,complaint_type_for_level_2,source_node_type,source_index,target_node_type,target_index,color,node
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9,NaN,complaint_type,0,process,17,"rgba(128, 128, 0, .6)",NaN
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958,NaN,complaint_type,1,process,18,"rgba(250, 190, 212, .6)",NaN
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35,NaN,complaint_type,1,process,19,"rgba(250, 190, 212, .6)",NaN
3,APPLIANCE,CONTACTED TENANT,5049,NaN,complaint_type,1,process,20,"rgba(250, 190, 212, .6)",NaN
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985,NaN,complaint_type,1,process,17,"rgba(250, 190, 212, .6)",NaN


### Plot the diagram

#### v1: White background

In [124]:

fig = go.Figure(data=[go.Sankey(
    textfont=dict(size=6),
    # Define nodes
    node = dict(
      pad = 50,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  sankey_info['data']['node']['label'],
      color =  'rgba(202, 190, 174, 1)'
    ),
    # Add links
    link = dict(
      source =  sankey_info['data']['link']['source'],
      target =  sankey_info['data']['link']['target'],
      value =  sankey_info['data']['link']['value'],
      color =  sankey_info['data']['link']['color'],
    label =  sankey_info['data']['link']['label'],
    hovertemplate = 'Complaint type: %{label} <br>Source: %{source.label} <br>Target: %{target.label}'
))])

fig.update_layout(title_text="NYC HPD 311 complaints resolution process<br>Data source: NYC Open Data Portal",
                  font=dict(size = 9, color = 'black'),
                  plot_bgcolor='white',
                  paper_bgcolor='white',
                 title_font_size = 15)

fig.add_annotation(
            x=0,
            y=-.1,
            text="Complaint type", showarrow=False, font = dict(size=12))

fig.add_annotation(
            x=.5,
            y=-.1,
            text="Resolution process", showarrow=False, font = dict(size=12))

fig.add_annotation(
            x=1,
            y=-.1,
            text="Resolution outcome", showarrow=False, font = dict(size=12))
fig.show()
fig.write_html('output/sankey_colored_by_complaint_type_white_background.html')

#### v2: Black background

In [123]:
#Black
fig = go.Figure(data=[go.Sankey(
    textfont=dict(size=6, color='black'),
    # Define nodes
    node = dict(
      pad = 50,
      thickness = 15,
      line = dict(color = "white", width = 0.5),
      label =  sankey_info['data']['node']['label'],
      color =  'rgba(225, 225, 225, 1)'
    ),
    # Add links
    link = dict(
      source =  sankey_info['data']['link']['source'],
      target =  sankey_info['data']['link']['target'],
      value =  sankey_info['data']['link']['value'],
      color =  sankey_info['data']['link']['color'],
    label =  sankey_info['data']['link']['label'],
    hovertemplate = 'Complaint type: %{label} <br>Source: %{source.label} <br>Target: %{target.label}'
))])

fig.update_layout(title_text="NYC HPD 311 complaints resolution process<br>Data source: NYC Open Data Portal",
                  font=dict(size = 6, color = 'white'),
                  plot_bgcolor='black',
                  paper_bgcolor='black',
                 title_font_size = 15)
fig.show()
fig.write_html('output/sankey_colored_by_complaint_type_black_background.html')

### Part 3: Create a 3rd sankey diagram with individual nodes by complaint type at process layer 
In the above diagrams the colors don't connect in layer 1 and layer 2 which makes it hard to read. This is an alternative that makes it continuous. Unfortunately plotly doesn't let you control the order of nodes or which labels to surpress, so it's unreadable when formatted this way.


### Source/node datatest defining nodes in second layer as pairwise combinations of the complaint type and process tokens

In [88]:
#First set of links is the same complaint_type/process set
display(df_by_complaint_process.head())

#Second set of links requires to group by complaint_type too
df_by_complaint_process_outcome = pd.DataFrame(to_analyze.groupby(['grouped_complaint_type','grouped_ultimate_process', 'grouped_ultimate_outcome'])['unique_key'].nunique()).reset_index()
df_by_complaint_process_outcome.columns = ['complaint_type_for_level_2', 'source', 'target', 'num_records']
display(df_by_complaint_process_outcome.head())

#Concatenate into one edges_df
edges_df = pd.concat([df_by_complaint_process, df_by_complaint_process_outcome], ignore_index = True, sort = False)
display(edges_df.head())

,source,target,num_records
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35
3,APPLIANCE,CONTACTED TENANT,5049
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985


,complaint_type_for_level_2,source,target,num_records
0,AGENCY,INSPECTED ORIGINAL PROPERTY,NO VIOLATIONS WERE ISSUED,6
1,AGENCY,INSPECTED ORIGINAL PROPERTY,VIOLATIONS WERE ISSUED,3
2,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,COMPLAINT STILL OPEN,30958
3,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),OUTCOME UNKNOWN CLOSED,35
4,APPLIANCE,CONTACTED TENANT,CONDITIONS CORRECTED,5049


,source,target,num_records,complaint_type_for_level_2
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9,NaN
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958,NaN
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35,NaN
3,APPLIANCE,CONTACTED TENANT,5049,NaN
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985,NaN


#### Define a node dict that separately defines the nodes in the process layer to be distinct (but grouped) according to complaint_type


In [89]:

#Separate out complaint_type
complaint_type_nodes = node_dict[node_dict['node_type'] == 'complaint_type']

#Separate out outcome
outcome_nodes = node_dict[node_dict['node_type'] == 'outcome']
# display(outcome_nodes)

#Get max of complaint_type nodes
max_val = complaint_type_nodes['node_index'].max()

#Define process nodes by complaint type
process_nodes = pd.DataFrame(df_by_complaint_process_outcome.groupby(['complaint_type_for_level_2', 'source'])['num_records'].nunique()).reset_index()[['complaint_type_for_level_2', 'source']]
process_nodes.sort_values('source', inplace = True)
process_nodes['node_index'] = range(max_val+1, max_val + process_nodes.shape[0]+1)
process_nodes.rename(columns = {'source':'node'}, inplace= True)
process_nodes['node_type'] = 'process'
# display(process_nodes)

#Get max of process nodes
max_val = process_nodes['node_index'].max()

#Finish node indexing
outcome_nodes['node_index'] = range(max_val+1, max_val + outcome_nodes.shape[0]+1)

#Concatenate nodes into dictionary
node_dict = pd.concat([complaint_type_nodes, process_nodes, outcome_nodes], ignore_index = True, sort = False)
display(node_dict)

/usr/local/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,node,node_type,node_index,complaint_type_for_level_2
0,AGENCY,complaint_type,0,NaN
1,APPLIANCE,complaint_type,1,NaN
2,CONSTRUCTION,complaint_type,2,NaN
3,DOOR/WINDOW,complaint_type,3,NaN
4,ELECTRIC,complaint_type,4,NaN
5,ELEVATOR,complaint_type,5,NaN
6,FLOORING/STAIRS,complaint_type,6,NaN
7,GENERAL,complaint_type,7,NaN
8,HEAT/HOT WATER,complaint_type,8,NaN
9,HPD LITERATURE REQUEST,complaint_type,9,NaN


#### Create groups of process nodes 
Plotly lets you specify groups, but this actually gets rid of the continuity so I don't use it in the graph

In [90]:
process_nodes = node_dict[node_dict['node_type']=='process']
grouped = process_nodes.groupby('node')
i=0
for group_name, group_df in grouped:
    display(group_name)
    display(group_df)
    if i==0:
        groups = [list(group_df['node_index'].values)]
    else:
        groups.append(list(group_df['node_index'].values))
    i+=1

'ATTEMPTED AND FAILED INSPECTION'

,node,node_type,node_index,complaint_type_for_level_2
17,ATTEMPTED AND FAILED INSPECTION,process,17,HEAT/HOT WATER
18,ATTEMPTED AND FAILED INSPECTION,process,18,FLOORING/STAIRS
19,ATTEMPTED AND FAILED INSPECTION,process,19,PLUMBING
20,ATTEMPTED AND FAILED INSPECTION,process,20,ELEVATOR
21,ATTEMPTED AND FAILED INSPECTION,process,21,PAINT/PLASTER
22,ATTEMPTED AND FAILED INSPECTION,process,22,ELECTRIC
23,ATTEMPTED AND FAILED INSPECTION,process,23,DOOR/WINDOW
24,ATTEMPTED AND FAILED INSPECTION,process,24,UNSANITARY CONDITION/SAFETY
25,ATTEMPTED AND FAILED INSPECTION,process,25,STRUCTURAL
26,ATTEMPTED AND FAILED INSPECTION,process,26,OUTSIDE BUILDING


'ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN)'

,node,node_type,node_index,complaint_type_for_level_2
31,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),process,31,FLOORING/STAIRS
32,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),process,32,CONSTRUCTION
33,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),process,33,APPLIANCE
34,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),process,34,PLUMBING
35,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),process,35,NONCONST
36,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),process,36,ELEVATOR
37,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),process,37,DOOR/WINDOW
38,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),process,38,UNSANITARY CONDITION/SAFETY
39,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),process,39,HEAT/HOT WATER
40,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),process,40,ELECTRIC


'BY INSPECTING ANOTHER APARTMENT'

...

,node,node_type,node_index,complaint_type_for_level_2
58,INSPECTED ORIGINAL PROPERTY,process,58,PAINT/PLASTER
59,INSPECTED ORIGINAL PROPERTY,process,59,PLUMBING
60,INSPECTED ORIGINAL PROPERTY,process,60,OUTSIDE BUILDING
61,INSPECTED ORIGINAL PROPERTY,process,61,STRUCTURAL
62,INSPECTED ORIGINAL PROPERTY,process,62,UNSANITARY CONDITION/SAFETY
63,INSPECTED ORIGINAL PROPERTY,process,63,NONCONST
64,INSPECTED ORIGINAL PROPERTY,process,64,AGENCY
65,INSPECTED ORIGINAL PROPERTY,process,65,ELEVATOR
66,INSPECTED ORIGINAL PROPERTY,process,66,HEAT/HOT WATER
67,INSPECTED ORIGINAL PROPERTY,process,67,VACANT APARTMENT


'LITERATURE PREPARATION'

,node,node_type,node_index,complaint_type_for_level_2
74,LITERATURE PREPARATION,process,74,HPD LITERATURE REQUEST


'RESOLUTION PROCESS UNKNOWN'

,node,node_type,node_index,complaint_type_for_level_2
75,RESOLUTION PROCESS UNKNOWN,process,75,UNSANITARY CONDITION/SAFETY
76,RESOLUTION PROCESS UNKNOWN,process,76,APPLIANCE
77,RESOLUTION PROCESS UNKNOWN,process,77,CONSTRUCTION
78,RESOLUTION PROCESS UNKNOWN,process,78,DOOR/WINDOW
79,RESOLUTION PROCESS UNKNOWN,process,79,PLUMBING
80,RESOLUTION PROCESS UNKNOWN,process,80,NONCONST
81,RESOLUTION PROCESS UNKNOWN,process,81,HPD LITERATURE REQUEST
82,RESOLUTION PROCESS UNKNOWN,process,82,ELEVATOR
83,RESOLUTION PROCESS UNKNOWN,process,83,PAINT/PLASTER
84,RESOLUTION PROCESS UNKNOWN,process,84,FLOORING/STAIRS


### Create container for diagram

In [91]:
#Initialize dictionary to hold data for sankey diagram
sankey_info = {}

#Assign indices to each node for source/target labeling
node_dict['node_index'] = range(node_dict.shape[0])
sankey_info['data'] = {}
sankey_info['data']['node'] = {}
sankey_info['data']['node']['label'] = list(node_dict['node'].values)

#Merge in the node indices to the edges_df

#Source indices
#Separate out first layer of edges
first_layer = edges_df[edges_df['complaint_type_for_level_2'].isna()]
# display(first_layer.head())

#Separate out second layer of edges
second_layer = edges_df[~edges_df['complaint_type_for_level_2'].isna()]
# display(second_layer.head())

#Merge in node indices to source by layer
first_layer_with_node_indices = first_layer.merge(node_dict[['node', 'node_type', 'node_index']], left_on='source', right_on = 'node', how = 'left')
first_layer_with_node_indices.rename(columns = {'node_index':'source_index', 'node_type':'source_node_type'}, inplace = True)
first_layer_with_node_indices.drop(columns = 'node', inplace = True)
# display(first_layer_with_node_indices.head())

second_layer_with_node_indices = second_layer.merge(node_dict, left_on=['source', 'complaint_type_for_level_2'], right_on = ['node', 'complaint_type_for_level_2'], how = 'left')
second_layer_with_node_indices.rename(columns = {'node_index':'source_index', 'node_type':'source_node_type'}, inplace = True)
second_layer_with_node_indices.drop(columns = 'node', inplace = True)
# display(second_layer_with_node_indices.head())

#Target
#Merge in node indices to source by layer
first_layer_with_node_indices = first_layer_with_node_indices.merge(node_dict, left_on=['target', 'source'], right_on = ['node', 'complaint_type_for_level_2'], how = 'left')
first_layer_with_node_indices.rename(columns = {'node_index':'target_index', 'node_type':'target_node_type', 'complaint_type_for_level_2_x':'complaint_type_for_level_2'}, inplace = True)
first_layer_with_node_indices.drop(columns = ['node', 'complaint_type_for_level_2_y'], inplace = True)

second_layer_with_node_indices = second_layer_with_node_indices.merge(node_dict[['node', 'node_type', 'node_index']], left_on='target', right_on = 'node', how = 'left')
second_layer_with_node_indices.rename(columns = {'node_index':'target_index', 'node_type':'target_node_type'}, inplace = True)
second_layer_with_node_indices.drop(columns = 'node', inplace = True)

#Concatenate
edges_with_node_indices = pd.concat([first_layer_with_node_indices, second_layer_with_node_indices], ignore_index = True, sort = False)
display(edges_with_node_indices.head())

# #Use assigned indices to define source/target lists for diagram
sankey_info['data']['link'] = {}
sankey_info['data']['link']['source'] = list(edges_with_node_indices['source_index'].values)
sankey_info['data']['link']['target'] = list(edges_with_node_indices['target_index'].values)
sankey_info['data']['link']['value'] = list(edges_with_node_indices['num_records'].values)

,source,target,num_records,complaint_type_for_level_2,source_node_type,source_index,target_node_type,target_index
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9,NaN,complaint_type,0,process,64
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958,NaN,complaint_type,1,process,27
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35,NaN,complaint_type,1,process,33
3,APPLIANCE,CONTACTED TENANT,5049,NaN,complaint_type,1,process,55
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985,NaN,complaint_type,1,process,68


#### Assign colors to nodes and links such that they are colored according to complaint type

In [92]:

#Define colors to use
twenty_distinct_colors = ['(230, 25, 75)', '(60, 180, 75)', '(255, 225, 25)', '(0, 130, 200)', '(245, 130, 48)', '(145, 30, 180)', '(70, 240, 240)', '(240, 50, 230)', '(210, 245, 60)', '(250, 190, 212)', '(0, 128, 128)', '(220, 190, 255)', '(170, 110, 40)', '(255, 250, 200)', '(128, 0, 0)', '(170, 255, 195)', '(128, 128, 0)', '(255, 215, 180)', '(0, 0, 128)', '(128, 128, 128)', '(255, 255, 255)', '(0, 0, 0)']

#Omit black and gray (so actually 18)
distinct_colors = ['(230, 25, 75)', '(60, 180, 75)', '(255, 225, 25)', '(0, 130, 200)', '(245, 130, 48)', '(145, 30, 180)', '(70, 240, 240)', '(240, 50, 230)', '(210, 245, 60)', '(250, 190, 212)', '(0, 128, 128)', '(220, 190, 255)', '(170, 110, 40)', '(255, 250, 200)', '(128, 0, 0)', '(170, 255, 195)', '(128, 128, 0)', '(255, 215, 180)', '(0, 0, 128)', '(255, 255, 255)']

#Randomly shuffle colors
random.shuffle(distinct_colors)

#Merge in colors to complaint dict
complaint_dict['color'] = distinct_colors[0:complaint_dict.shape[0]]

#Use complaint dict to set edge colors
#Separate by layer
source_complaint_type = edges_with_node_indices[edges_with_node_indices['source_node_type'] == 'complaint_type']
source_process = edges_with_node_indices[edges_with_node_indices['source_node_type'] == 'process']

#Merge in colors to complaint type source
source_complaint_type_with_colors = source_complaint_type.merge(complaint_dict[['node', 'color']], left_on = 'source', right_on = 'node', how = 'left')
source_complaint_type_with_colors.drop(columns = 'node', inplace = True)
display(source_complaint_type_with_colors.head())

#Use complaint type to color second layer
source_process_with_colors = source_process.merge(complaint_dict[['node', 'color']], left_on = 'complaint_type_for_level_2', right_on = 'node', how = 'left')
display(source_process_with_colors.head())

#Concatenate edges back together
edges_with_colors = pd.concat([source_complaint_type_with_colors, source_process_with_colors], ignore_index = True, sort = False)

#Add opacity for edges and nodes
edges_with_colors['color'] = edges_with_colors['color'].apply(lambda color: 'rgba' + color.split(')')[0] + ', .6)')

display(edges_with_colors.head())

#Define node/edge colors
sankey_info['data']['node']['color'] = ["rgba(255, 255, 255, 1)" for i in range(len(sankey_info['data']['node']['label']))]
sankey_info['data']['link']['color'] = list(edges_with_colors['color'].values)


,source,target,num_records,complaint_type_for_level_2,source_node_type,source_index,target_node_type,target_index,color
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9,NaN,complaint_type,0,process,64,"(0, 130, 200)"
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958,NaN,complaint_type,1,process,27,"(170, 110, 40)"
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35,NaN,complaint_type,1,process,33,"(170, 110, 40)"
3,APPLIANCE,CONTACTED TENANT,5049,NaN,complaint_type,1,process,55,"(170, 110, 40)"
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985,NaN,complaint_type,1,process,68,"(170, 110, 40)"


,source,target,num_records,complaint_type_for_level_2,source_node_type,source_index,target_node_type,target_index,node,color
0,INSPECTED ORIGINAL PROPERTY,NO VIOLATIONS WERE ISSUED,6,AGENCY,process,64,outcome,90,AGENCY,"(0, 130, 200)"
1,INSPECTED ORIGINAL PROPERTY,VIOLATIONS WERE ISSUED,3,AGENCY,process,64,outcome,91,AGENCY,"(0, 130, 200)"
2,ATTEMPTED AND FAILED INSPECTION,COMPLAINT STILL OPEN,30958,APPLIANCE,process,27,outcome,92,APPLIANCE,"(170, 110, 40)"
3,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),OUTCOME UNKNOWN CLOSED,35,APPLIANCE,process,33,outcome,93,APPLIANCE,"(170, 110, 40)"
4,CONTACTED TENANT,CONDITIONS CORRECTED,5049,APPLIANCE,process,55,outcome,94,APPLIANCE,"(170, 110, 40)"


,source,target,num_records,complaint_type_for_level_2,source_node_type,source_index,target_node_type,target_index,color,node
0,AGENCY,INSPECTED ORIGINAL PROPERTY,9,NaN,complaint_type,0,process,64,"rgba(0, 130, 200, .6)",NaN
1,APPLIANCE,ATTEMPTED AND FAILED INSPECTION,30958,NaN,complaint_type,1,process,27,"rgba(170, 110, 40, .6)",NaN
2,APPLIANCE,ATTEMPTED INSPECTION OF ORIGINAL PROPERTY (SUCCESS UNKNOWN),35,NaN,complaint_type,1,process,33,"rgba(170, 110, 40, .6)",NaN
3,APPLIANCE,CONTACTED TENANT,5049,NaN,complaint_type,1,process,55,"rgba(170, 110, 40, .6)",NaN
4,APPLIANCE,INSPECTED ORIGINAL PROPERTY,80985,NaN,complaint_type,1,process,68,"rgba(170, 110, 40, .6)",NaN


### Plot figure

In [93]:
fig = go.Figure(data=[go.Sankey(
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "white", width = 0.5),
      label =  sankey_info['data']['node']['label'],
      color =  sankey_info['data']['node']['color'],
#     groups = groups This just makes it look like the one above
    ),
    # Add links
    link = dict(
      source =  sankey_info['data']['link']['source'],
      target =  sankey_info['data']['link']['target'],
      value =  sankey_info['data']['link']['value'],
      color =  sankey_info['data']['link']['color']
))])

fig.update_layout(title_text="NYC HPD 311 complaints resolution process<br>Data source: NYC Open Data Portal",
                  font=dict(size = 7, color = 'white'),
                  plot_bgcolor='black',
                  paper_bgcolor='black')
fig.show()
fig.write_html('output/sankey_colored_by_complaint_type_nodes_separate.html')